In [152]:
import json
from urllib.request import urlopen
import pandas as pd
from pulp import *

In [185]:
#Set number of available players

formations = [
    {'GKP': 1, 'DEF': 3, 'MID': 4, 'FWD': 3},
    {'GKP': 1, 'DEF': 3, 'MID': 5, 'FWD': 2},
    {'GKP': 1, 'DEF': 4, 'MID': 5, 'FWD': 1},
    {'GKP': 1, 'DEF': 4, 'MID': 4, 'FWD': 2},
    {'GKP': 1, 'DEF': 4, 'MID': 3, 'FWD': 3} 
]

#Set number of substitutes 

sub_formations= [
    {'GKP': 1, 'DEF': 2, 'MID': 1, 'FWD': 0},
    {'GKP': 1, 'DEF': 2, 'MID': 0, 'FWD': 1},
    {'GKP': 1, 'DEF': 1, 'MID': 0, 'FWD': 2},
    {'GKP': 1, 'DEF': 1, 'MID': 1, 'FWD': 1},
    {'GKP': 1, 'DEF': 1, 'MID': 2, 'FWD': 0}
]

#Set Budget

BUDGET = 83
SUB_BUDGET = 17
PLAYER_TEAM_LIMIT = 3

In [186]:
class Dataset():
    
    def get_url(self, key, Id=None):
        base_url = 'https://fantasy.premierleague.com/drf'
        api = {
            'All': '/bootstrap-static',
            'Player': '/element-summary/{}',
            'Team':'/my-team/48049/{}',
            '1':'/bootstrap',
            '2':'/bootstrap-static',
            '3':'/bootstrap-dynamic',
            '4':'/events',
            'player_stats':'/elements',
            'player_positions':'/element-types',
            '7':'/fixtures',
            'teams':'/teams',
            '9':'/region',
            '10':'/transfers',
            '11':'/entry/{}',
            '12':'/entries',
            '13':'/my-team/{}',
            '14':'/leagues-entered/{}',
            '15':'/leagues-classic/{}',
            '16':'/leagues-classic-standings/{}',
            '17':'/leagues-h2h/{}',
            '18':'/leagues-h2h-standings/{}'
        }

        if Id is None:
            url = ''.join([base_url, api[key]])
        else:
            api_extension = api[key]
            api_extension = api_extension.format(Id)
            url = ''.join([base_url, api_extension])
        return url

    def pull_data(self, key, Id=None):
        json_data = json.load(urlopen(self.get_url(key, Id)))
        df = pd.read_json(json.dumps(json_data))
        return df

    def get_player_data(self, active=False):
        player_stats = self.pull_data('player_stats')
        player_positions = self.pull_data('player_positions')[['id', 'singular_name_short']]
        player_teams = self.pull_data('teams')[['code', 'name']]
        
        player_table = player_stats.merge(player_positions, left_on='element_type', right_on='id')
        player_table = player_table.merge(player_teams, left_on='team_code', right_on='code')
        player_table['now_cost'] = player_table['now_cost']/10
        if active:
            player_table = player_table.loc[player_table['chance_of_playing_next_round'] == 100]
            player_table = player_table.reset_index()
        return player_table
    
    def player_model_data(self, active=True):
        
        active_players = self.get_player_data(active)
        player_pool = active_players[['web_name', 'singular_name_short', 'now_cost', 'total_points', 'name']]
        player_pool.columns = ['web_name', 'position', 'cost', 'total_points', 'team_name']
        
        def pd_to_dict(key, value):
            return list(player_pool[[key, value]].set_index(key).to_dict().values())[0]
        
        player_names = set(player_pool['web_name'])
        team_names = set(player_pool['team_name'])
        position_names = set(player_pool['position'])
        
        points = pd_to_dict('web_name', 'total_points')
        costs = pd_to_dict('web_name', 'cost')
        player_team = pd_to_dict('web_name', 'team_name')
        player_position = pd_to_dict('web_name', 'position')
        
        teams = {team:[player for player in player_names if player_team[player]==team] for team in team_names}
        positions = {pos:[player for player in player_names if player_position[player]==pos] for pos in position_names}

        return (player_names, points, costs, teams, positions)
        

In [187]:
class TeamSolutions():
    def __init__(self, data, formations, budgets):
        self.formations = formations
        self.budgets = budgets
        self.players, self.points, self.costs, self.teams, self.positions = data
        
    def solve_problem(self, formation, budget):
        
        player_vars = LpVariable.dicts("players", ((player) for player in self.players), cat='Binary')
        model = LpProblem('Naive_total_score_max', LpMaximize)
        
        #Set Objective
        model += (
            lpSum(
                [player_vars[player] * self.points[player] for player in self.players]
            )
        )
        
        #Constraints
        model += lpSum(player_vars[player] * self.costs[player] for player in self.players) <= budget
        
        for players in self.teams.values():
            model += lpSum([player_vars[player] for player in players]) <= PLAYER_TEAM_LIMIT
            
        for position, players in self.positions.items():
            print(position, formation[position])
            model += lpSum([player_vars[player] for player in players]) == formation[position]
        
        model.solve()
        return model, player_vars
    
    def summarise(self, model, player_vars, formation=None):
        costs = []
        players = []
        positions = []
        teams = []
        points = []
        for var in player_vars:
            var_value = player_vars[var].varValue
            if var_value == 1.0:
                players.append(var)
                costs.append(self.costs[var])
                points.append(self.points[var])
                positions.append([position for position, players in self.positions.items() if var in players][0])
                teams.append([team for team, players in self.teams.items() if var in players][0])
        df= pd.DataFrame({'Player':players, 'Position':positions, 'Cost': costs, 'Team': teams, 'Points':points})
        if formation:
            print(formation.items())
        print(df)
        print('Points: {}'.format(value(model.objective)))

    def solve_for_all_formations(self):
        possible_teams = []
        for formation in self.formations:
            possible_team, players = self.solve_problem(formation, self.budgets)
            team_summary = self.summarise(possible_team, players, formation)
            
            

In [188]:
datasets = Dataset()

In [189]:
active_player_data = datasets.player_model_data()
all_player_data = datasets.player_model_data(False)

In [190]:
Team = TeamSolutions(player_data, formations, BUDGET)
Subs = TeamSolutions(all_player_data, sub_formations, SUB_BUDGET)

In [191]:
Team.solve_for_all_formations()

FWD 3
MID 4
GKP 1
DEF 3
dict_items([('GKP', 1), ('DEF', 3), ('MID', 4), ('FWD', 3)])
    Cost       Player  Points Position         Team
0   11.5       Agüero      82      FWD     Man City
1    5.0     Dubravka      51      GKP    Newcastle
2    6.2       Fraser      72      MID  Bournemouth
3    5.9      Jiménez      58      FWD       Wolves
4    7.0  Richarlison      62      MID      Everton
5    5.0        Keane      57      DEF      Everton
6   11.3     Sterling      95      MID     Man City
7    6.1      Laporte      64      DEF     Man City
8   10.9   Aubameyang      75      FWD      Arsenal
9    7.4   Sigurdsson      72      MID      Everton
10   6.0      Rüdiger      58      DEF      Chelsea
Points: 746.0
FWD 2
MID 5
GKP 1
DEF 3
dict_items([('GKP', 1), ('DEF', 3), ('MID', 5), ('FWD', 2)])
    Cost       Player  Points Position         Team
0   11.5       Agüero      82      FWD     Man City
1    5.0     Dubravka      51      GKP    Newcastle
2    6.2       Fraser      72      M

In [192]:
Subs.solve_for_all_formations()

FWD 0
MID 1
GKP 1
DEF 2
dict_items([('GKP', 1), ('DEF', 2), ('MID', 1), ('FWD', 0)])
   Cost       Player  Points Position            Team
0   4.5    Hennessey      55      GKP  Crystal Palace
1   3.8      Peltier       1      DEF         Cardiff
2   4.5      Billing      33      MID    Huddersfield
3   4.2  Wan-Bissaka      42      DEF  Crystal Palace
Points: 131.0
FWD 1
MID 0
GKP 1
DEF 2
dict_items([('GKP', 1), ('DEF', 2), ('MID', 0), ('FWD', 1)])
   Cost     Player  Points Position            Team
0   3.9  Stankovic       6      DEF    Huddersfield
1   4.5  Hennessey      55      GKP  Crystal Palace
2   4.3  Schindler      45      DEF    Huddersfield
3   4.3     Kamara       6      FWD          Fulham
Points: 112.0
FWD 2
MID 0
GKP 1
DEF 1
dict_items([('GKP', 1), ('DEF', 1), ('MID', 0), ('FWD', 2)])
   Cost     Player  Points Position            Team
0   3.9  Stankovic       6      DEF    Huddersfield
1   4.5  Hennessey      55      GKP  Crystal Palace
2   4.3     Quaner       0     